In [1]:
from sagemaker import RandomCutForest


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import json

In [3]:
aws_role = get_execution_role()


sm_role = sagemaker.get_execution_role()

aws_region = boto3.Session().region_name
sm_session = sagemaker.Session()

print(f"Using Role: {sm_role}")
s3_bucket = sm_session.default_bucket()
s3_bucket

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Using Role: arn:aws:iam::287758680514:role/service-role/AmazonSageMaker-ExecutionRole-20231027T004238


'sagemaker-us-east-1-287758680514'

In [12]:
import pandas as pd
url = 'https://raw.githubusercontent.com/numenta/NAB/master/data/realKnownCause/ambient_temperature_system_failure.csv'
df = pd.read_csv(url, sep=',')

In [13]:
df

,timestamp,value
0,2013-07-04 00:00:00,69.880835
1,2013-07-04 01:00:00,71.220227
2,2013-07-04 02:00:00,70.877805
3,2013-07-04 03:00:00,68.959400
4,2013-07-04 04:00:00,69.283551
...,...,...
7262,2014-05-28 11:00:00,72.370206
7263,2014-05-28 12:00:00,72.172956
7264,2014-05-28 13:00:00,72.046565
7265,2014-05-28 14:00:00,71.825226


In [16]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [18]:
df['year'] = df['timestamp'].apply(lambda x: x.year)

In [20]:
df['month'] = df['timestamp'].apply(lambda x: x.month)

In [22]:
df['day'] = df['timestamp'].apply(lambda x: x.day)

In [26]:
df['hour'] = df['timestamp'].apply(lambda x: x.hour)

In [27]:
df['minute'] = df['timestamp'].apply(lambda x: x.minute)

In [28]:
df

,timestamp,value,year,month,day,minute,hour
0,2013-07-04 00:00:00,69.880835,2013,7,4,0,0
1,2013-07-04 01:00:00,71.220227,2013,7,4,0,1
2,2013-07-04 02:00:00,70.877805,2013,7,4,0,2
3,2013-07-04 03:00:00,68.959400,2013,7,4,0,3
4,2013-07-04 04:00:00,69.283551,2013,7,4,0,4
...,...,...,...,...,...,...,...
7262,2014-05-28 11:00:00,72.370206,2014,5,28,0,11
7263,2014-05-28 12:00:00,72.172956,2014,5,28,0,12
7264,2014-05-28 13:00:00,72.046565,2014,5,28,0,13
7265,2014-05-28 14:00:00,71.825226,2014,5,28,0,14


In [29]:
anomaly_points = [
        ["2013-12-10 06:25:00.000000","2013-12-12 05:35:00.000000"],
        ["2013-12-15 17:50:00.000000","2013-12-17 17:00:00.000000"],
        ["2014-01-27 14:20:00.000000","2014-01-29 13:30:00.000000"],
        ["2014-02-07 14:55:00.000000","2014-02-09 14:05:00.000000"]
]

In [30]:
anomaly_points

[['2013-12-10 06:25:00.000000', '2013-12-12 05:35:00.000000'],
 ['2013-12-15 17:50:00.000000', '2013-12-17 17:00:00.000000'],
 ['2014-01-27 14:20:00.000000', '2014-01-29 13:30:00.000000'],
 ['2014-02-07 14:55:00.000000', '2014-02-09 14:05:00.000000']]

In [43]:
df['anomaly'] = 0

In [44]:
for start,end in anomaly_points:
    print (start, end)
    df.loc[  ((df['timestamp'] >= start) & (df['timestamp'] <= end)), ['anomaly']] = 1  

2013-12-10 06:25:00.000000 2013-12-12 05:35:00.000000
2013-12-15 17:50:00.000000 2013-12-17 17:00:00.000000
2014-01-27 14:20:00.000000 2014-01-29 13:30:00.000000
2014-02-07 14:55:00.000000 2014-02-09 14:05:00.000000


In [53]:
df[df['anomaly'] == 1].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 190 entries, 3420 to 4891
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  190 non-null    datetime64[ns]
 1   value      190 non-null    float64       
 2   year       190 non-null    int64         
 3   month      190 non-null    int64         
 4   day        190 non-null    int64         
 5   minute     190 non-null    int64         
 6   hour       190 non-null    int64         
 7   anomaly    190 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(6)
memory usage: 13.4 KB


In [54]:
df.index = df['timestamp']

In [56]:
df.drop(['timestamp'], axis=1, inplace=True)

In [57]:
df

,value,year,month,day,minute,hour,anomaly
timestamp,,,,,,,
2013-07-04 00:00:00,69.880835,2013,7,4,0,0,0
2013-07-04 01:00:00,71.220227,2013,7,4,0,1,0
2013-07-04 02:00:00,70.877805,2013,7,4,0,2,0
2013-07-04 03:00:00,68.959400,2013,7,4,0,3,0
2013-07-04 04:00:00,69.283551,2013,7,4,0,4,0
...,...,...,...,...,...,...,...
2014-05-28 11:00:00,72.370206,2014,5,28,0,11,0
2014-05-28 12:00:00,72.172956,2014,5,28,0,12,0
2014-05-28 13:00:00,72.046565,2014,5,28,0,13,0


In [ ]:
!conda install -y holoviews

Retrieving notices: ...working... done
done
Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::bleach==6.1.0=pyhd8ed1ab_0
  - pytorch/linux-64::torch-model-archiver==0.8.2=py310_0
  - pytorch/linux-64::torch-workflow-archiver==0.2.11=py310_0
  - pytorch/linux-64::torchserve==0.8.2=py310_0
  - conda-forge/linux-64::contourpy==1.1.1=py310hd41b1e2_1
  - conda-forge/noarch::imageio==2.31.5=pyh8c1a49c_0
  - conda-forge/linux-64::onnx==1.13.1=py310ha3deec4_2
  - conda-forge/linux-64::scipy==1.11.3=py310hb13e2d6_1
  - conda-forge/noarch::bokeh==3.3.0=pyhd8ed1ab_0
  - conda-forge/noarch::ipykernel==6.26.0=pyhf8b6a83_0
  - conda-forge/linux-64::matplotlib-base==3.8.0=py310h62c0568_2
  - conda-forge/noarch::nbconvert-core==7.9.2=pyhd8ed1ab_0
  - conda-forge/noarch::patsy==0.5.3=pyhd8ed1ab_0
  - conda-forge/noarch::jupyter_server==2.9.1=pyhd8ed1ab_0
  - conda-forge/noarch::

In [ ]:
import seaborn as sns
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

In [5]:
prefix='randomCut'
rcf = RandaomCutForest(role=sm_role,
                      train_instance_count=1,
                      train_instance_type='ml.m4.xlarge',
                      data_location='s3://{s3_bucket}/{prefix}/',
                      output_path='s3://{s3_bucket}/{prefix}/output')

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/SageMaker/.xdg/config/sagemaker/config.yaml


In [6]:
rcf.fit(rcf.record_set(train_data))

NameError: name 'train_data' is not defined